# Portfolio Tracker

This notebook helps track and analyze an investment portfolio.

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# Step 2: Load Portfolio Holdings CSV
holdings = pd.read_csv('holdings.csv')
display(holdings.head()) 

In [ ]:
# Step 3: Fetch Current Prices
def fetch_current_prices(tickers):
    # Download latest prices
    data = yf.download(tickers, period='1d', progress=False)
    return data['Close'].iloc[-1]

tickers = holdings['Ticker'].tolist()
current_prices = fetch_current_prices(tickers)
holdings['Current Price'] = holdings['Ticker'].map(current_prices)
holdings['Current Value'] = holdings['Shares'] * holdings['Current Price']

display(holdings.head())

In [ ]:
# Step 4: Portfolio Analysis
portfolio_value = holdings['Current Value'].sum()

holdings['Portfolio Weight'] = holdings['Current Value'] / portfolio_value
holdings['Cost Basis'] = holdings['Shares'] * holdings['Purchase Price']
holdings['Unrealized Gain'] = holdings['Current Value'] - holdings['Cost Basis']
holdings['Total Return %'] = (holdings['Unrealized Gain'] / holdings['Cost Basis']) * 100
display(holdings)

In [ ]:
# Step 5: Pie Chart of Portfolio Allocation
fig = px.pie(holdings, values='Portfolio Weight', names='Ticker', title='Portfolio Allocation')
fig.show()

In [ ]:
# Step 6: Benchmark Comparison (S&P 500)
# Pull S&P 500 historical prices from the date of first purchase
earliest_date = holdings['Purchase Date'].min()
spy = yf.download('^GSPC', start=earliest_date, progress=False)['Close']

# Calculate cumulative returns
spy_returns = spy.pct_change().cumsum()

# Basic Line Graph
plt.figure(figsize=(10,6))
spy_returns.plot(label='S&P 500')
plt.title('Benchmark Comparison vs. S&P 500')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.show()